"""
Notebook para análises dos dados de filmes
Camada Gold: Dados prontos para consumo e tomada de decisão
"""

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

In [24]:
# Caminhos
SILVER_DIR = Path("../datalake/silver")
GOLD_DIR = Path("../datalake/gold")

# Carregar dados da camada silver 
try:
    df = pd.read_parquet(SILVER_DIR / "movies_processed.parquet") 
    print(f"Dados carregados: {len(df)} filmes")
    print(f"Período: {df['ano'].min()} - {df['ano'].max()}")
except FileNotFoundError:
    print("Arquivo Parquet não encontrado na camada silver")
    exit()


Dados carregados: 4499 filmes
Período: 1915 - 2005


In [25]:
def analise_exploratoria(df):
    """Análise exploratória dos dados"""
    print("\nANÁLISE EXPLORATÓRIA")
    
    # Informações básicas
    print(f"Total de filmes: {len(df)}")
    print(f"Filme mais antigo: {df.loc[df['ano'].idxmin(), 'filme']} ({df['ano'].min()})")
    print(f"Filme mais recente: {df.loc[df['ano'].idxmax(), 'filme']} ({df['ano'].max()})")
    print(f"Span temporal: {df['ano'].max() - df['ano'].min()} anos")
    
    # Estatísticas por década
    df['decada'] = (df['ano'] // 10) * 10
    decadas = df['decada'].value_counts().sort_index()
    print(f"\nFilmes por década:")
    for decada, count in decadas.items():
        print(f"  {int(decada)}s: {count} filmes")

    # Quantidade de filmes por ano
    filmes_por_ano = df['ano'].value_counts().sort_index()
    print(f"\nFilmes por ano:")
    for ano, count in filmes_por_ano.items():
        print(f"  {int(ano)}: {count} filmes")
    
    return df

df = analise_exploratoria(df)


ANÁLISE EXPLORATÓRIA
Total de filmes: 4499
Filme mais antigo: Les Vampires (1915)
Filme mais recente: 7 Seconds (2005)
Span temporal: 90 anos

Filmes por década:
  1910s: 6 filmes
  1920s: 26 filmes
  1930s: 61 filmes
  1940s: 80 filmes
  1950s: 148 filmes
  1960s: 236 filmes
  1970s: 321 filmes
  1980s: 534 filmes
  1990s: 1306 filmes
  2000s: 1781 filmes

Filmes por ano:
  1915: 1 filmes
  1916: 2 filmes
  1917: 1 filmes
  1918: 2 filmes
  1920: 3 filmes
  1921: 4 filmes
  1922: 1 filmes
  1924: 2 filmes
  1925: 4 filmes
  1926: 1 filmes
  1927: 6 filmes
  1928: 3 filmes
  1929: 2 filmes
  1930: 6 filmes
  1931: 2 filmes
  1932: 7 filmes
  1933: 3 filmes
  1934: 8 filmes
  1935: 4 filmes
  1936: 7 filmes
  1937: 4 filmes
  1938: 6 filmes
  1939: 14 filmes
  1940: 9 filmes
  1941: 12 filmes
  1942: 9 filmes
  1943: 8 filmes
  1944: 11 filmes
  1945: 9 filmes
  1946: 9 filmes
  1947: 6 filmes
  1948: 3 filmes
  1949: 4 filmes
  1950: 9 filmes
  1951: 15 filmes
  1952: 17 filmes
  1953

In [26]:
def categorizar_filmes(df):
    """Categorizar filmes por tipo e gênero"""
    print("CATEGORIZAÇÃO")
    print("-" * 28)
    
    # Detectar documentários
    df['is_documentary'] = df['filme'].str.contains(
        r'documentary|nature|review|live|bonus', case=False, na=False
    )
    
    # Detectar sequels/series
    df['is_sequel'] = df['filme'].str.contains(
        r'\b(2|3|4|II|III|IV)\b|part|volume', case=False, na=False
    )
    
    # Detectar animações
    df['is_animation'] = df['filme'].str.contains(
        r'man|cartoon|animated', case=False, na=False
    )
    
    # Categorias principais
    def get_categoria(row):
        if row['is_documentary']:
            return 'Documentário'
        elif row['is_animation']:
            return 'Animação'
        elif row['is_sequel']:
            return 'Sequel/Série'
        else:
            return 'Filme'
    
    df['categoria'] = df.apply(get_categoria, axis=1)
    
    # Estatísticas por categoria
    categoria_stats = df['categoria'].value_counts()
    print("Distribuição por categoria:")
    for cat, count in categoria_stats.items():
        pct = (count / len(df)) * 100
        print(f"  {cat}: {count} ({pct:.1f}%)")
    
    return df

df = categorizar_filmes(df)

CATEGORIZAÇÃO
----------------------------
Distribuição por categoria:
  Filme: 3856 (85.7%)
  Sequel/Série: 294 (6.5%)
  Animação: 185 (4.1%)
  Documentário: 164 (3.6%)


In [ ]:
# Diretório onde você quer salvar os arquivos 
gold_path = Path("../datalake/gold/movies_by_category")
gold_path.mkdir(parents=True, exist_ok=True)

# Cria um DataFrame por categoria e salva
categorias = df["categoria"].unique()

for cat in categorias:
    df_cat = df[df["categoria"] == cat].copy()

    # Seleciona apenas as colunas desejadas
    df_cat = df_cat[["id", "filme", "ano"]]

    
    # Alterado para salvar como .parquet
    filename = cat.lower().replace("/", "_").replace(" ", "_") + ".parquet" 
    filepath = gold_path / filename
    
    df_cat.to_parquet(filepath, index=False) # Alterado para to_parquet
    print(f"Arquivo salvo: {filepath}")

Arquivo salvo: ../datalake/gold/movies_by_category/filme.parquet
Arquivo salvo: ../datalake/gold/movies_by_category/documentário.parquet
Arquivo salvo: ../datalake/gold/movies_by_category/animação.parquet
Arquivo salvo: ../datalake/gold/movies_by_category/sequel_série.parquet
